In [2]:
import pandas as pd
from catboost import CatBoostClassifier

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

import optuna

from giskard import Dataset, Model, scan, testing
import pickle

from minio import Minio
from minio.error import S3Error
import warnings

# import clickhouse_connect
import psycopg2

from functools import reduce

import clickhouse_connect

import mlflow 
import mlflow.catboost
import mlflow.sklearn
import mlflow.data
from mlflow.models import infer_signature

from datetime import datetime

# Env variables

In [3]:
from dotenv import load_dotenv
import os

load_dotenv()

SPARK_COMPAT_VERSION = os.getenv('SPARK_COMPAT_VERSION')
SCALA_COMPAT_VERSION = os.getenv('SCALA_COMPAT_VERSION')
CATBOOST_SPARK_VERSION = os.getenv('CATBOOST_SPARK_VERSION')
DB_HOST = os.getenv('DB_HOST')
POSTGRESQL_PORT = os.getenv('POSTGRESQL_PORT')
CLICKHOUSE_PORT = os.getenv('CLICKHOUSE_PORT')
DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_NAME = os.getenv('DB_NAME')

ACCESS_KEY=os.getenv('MINIO_ROOT_USER')
SECRET_KEY=os.getenv('MINIO_ROOT_PASSWORD')

# Set feature and target columns

In [16]:
COLUMN_TYPES = {
    'age': 'numeric',
    'sex': 'category',
    'job': 'category',
    'housing': 'category',
    'credit_amount': 'numeric',
    'duration': 'numeric'
}

TARGET_COLUMN_NAME = 'default'
FEATURE_COLUMNS = [i for i in COLUMN_TYPES.keys()]
FEATURE_TYPES = {i: COLUMN_TYPES[i] for i in COLUMN_TYPES if i != TARGET_COLUMN_NAME}

COLUMNS_TO_SCALE = [key for key in COLUMN_TYPES.keys() if COLUMN_TYPES[key] == "numeric"]
COLUMNS_TO_ENCODE = [key for key in COLUMN_TYPES.keys() if COLUMN_TYPES[key] == "category"]

# Connect to db (data) and get data

In [17]:
# client = clickhouse_connect.get_client(host = CLICKHOUSE_HOST, 
#                                        port = CLICKHOUSE_PORT, 
#                                        user = CLICKHOUSE_USER, 
#                                        password = CLICKHOUSE_PASSWORD)
# query = fr'''
# select {reduce(lambda a,b: a + ', ' + b, FEATURE_COLUMNS)}
# from credit.credit
# '''
# X = pd.DataFrame(client.query(query).named_results())
# X

job_list = {
    0: 'unskilled and non-resident', 
    1: 'unskilled and resident', 
    2: 'skilled', 
    3: 'highly skilled'
}


## Postgresql ==============================================
conn = psycopg2.connect(dbname='credit',
                                user=DB_USER,
                                password=DB_PASSWORD,
                                host='localhost',
                                port=POSTGRESQL_PORT)
cur = conn.cursor()
cur.execute(f"SELECT {reduce(lambda a,b: a + ', ' + b, FEATURE_COLUMNS)} FROM credit;")
X = (
    pd
    .DataFrame(cur.fetchall(), columns=FEATURE_COLUMNS)
    .assign(job = lambda x: x['job'].apply(lambda x: job_list[x]))
)
conn.commit()
conn.close()


conn = psycopg2.connect(dbname='credit',
                                user=DB_USER,
                                password=DB_PASSWORD,
                                host='localhost',
                                port=POSTGRESQL_PORT)
cur = conn.cursor()
cur.execute(f'SELECT cr."{TARGET_COLUMN_NAME}" FROM credit cr;')
y = [x[0] for x in cur.fetchall()]
conn.commit()
conn.close()

df = X.join(pd.DataFrame(y, columns = [TARGET_COLUMN_NAME]))
#=================================================================


# MLflow connection 

In [6]:
warnings.filterwarnings('ignore')
mlflow.set_tracking_uri('http://localhost:5000/')
print("URI", mlflow.get_tracking_uri())

URI http://localhost:5000/


# Model creation

In [7]:
numeric_transformer = Pipeline(steps = [
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])
categorical_transformer = Pipeline(steps = [
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])
# 
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, COLUMNS_TO_SCALE),
        ("cat", categorical_transformer, COLUMNS_TO_ENCODE)
    ]
)

In [18]:
X_preproccessed = preprocessor.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size = 0.25,
    random_state=42
)

### Optimize hyperparams (optuna)

In [19]:
def objective(trial):    
    # CatBoostClassifier hyperparams
    params = {
        "objective": trial.suggest_categorical("objective", ["Logloss", "CrossEntropy"]),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "depth": trial.suggest_int("depth", 1, 12),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
        ),
        "used_ram_limit": "3gb",
    }
    with mlflow.start_run(run_name = f'run_{datetime.now().strftime('%d.%m.%Y_%H:%M:%S')}', nested=True):
        
        # model
        mlflow.log_params(params)
        estimator = CatBoostClassifier(**params, verbose=False)
        
        recall = cross_val_score(estimator, X_preproccessed, y, cv=5, scoring= 'recall').mean()
        mlflow.log_metric('Recall', recall) 
        return recall


experiment_name = f'credit_pred_{datetime.now().strftime('01.%m.%Y')}'
try:
    mlflow.create_experiment(experiment_name)
except:
    pass
mlflow.set_experiment(experiment_name)

with mlflow.start_run(run_name = f'params_opt_{datetime.now().strftime('%d.%m.%Y_%H:%M')}') as run:
    study = optuna.create_study(direction="minimize", study_name=f"params_opt_{datetime.now().strftime('%Y%m%d-%H%M%S')}")

    # Hyperparams searching
    study.optimize(objective, n_trials=4)
    
    # best result is
    params = study.best_params

    estimator = CatBoostClassifier(**params, verbose=False)  
    catboostclassifier = Pipeline(steps = [
        ("preprocessor", preprocessor),
        ("classifier", estimator)
    ])
    # catboostclassifier = CatBoostClassifier(**params, verbose=False)
    catboostclassifier.fit(X_train, y_train)
    
    pred_test = catboostclassifier.predict(X_test)
    signature = infer_signature(X_test, pred_test)

    
    metrics = {'accuracy': accuracy_score(pred_test, y_test),
                'precision': precision_score(pred_test, y_test),
                'recall': recall_score(pred_test, y_test),
                'f1': f1_score(pred_test, y_test),
                'roc_auc': roc_auc_score(y_test, catboostclassifier.predict_proba(X_test)[:,1])
              }

    
    input_example = X_train.iloc[[0], :]
    mlflow.models.infer_signature(input_example, 0, params)
    mlflow.log_params(params)
    mlflow.log_metrics(metrics)

    dataset = mlflow.data.from_pandas(df, name='german_credit', targets='default')
    mlflow.log_input(dataset)

    mlflow.sklearn.log_model(sk_model=catboostclassifier, 
                              artifact_path='catboostclassifier', 
                              signature=signature,
                              input_example=input_example
                             )

[I 2025-05-13 20:58:13,339] A new study created in memory with name: params_opt_20250513-205813
[I 2025-05-13 20:58:40,177] Trial 0 finished with value: 0.2733333333333333 and parameters: {'objective': 'CrossEntropy', 'colsample_bylevel': 0.0781534979726857, 'depth': 10, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bernoulli'}. Best is trial 0 with value: 0.2733333333333333.


🏃 View run run_13.05.2025_20:58:13 at: http://localhost:5000/#/experiments/3/runs/e65530c6930145b8895826c962b5a65d
🧪 View experiment at: http://localhost:5000/#/experiments/3


[I 2025-05-13 20:58:48,708] Trial 1 finished with value: 0.2733333333333333 and parameters: {'objective': 'CrossEntropy', 'colsample_bylevel': 0.09144275187578509, 'depth': 7, 'boosting_type': 'Ordered', 'bootstrap_type': 'MVS'}. Best is trial 0 with value: 0.2733333333333333.


🏃 View run run_13.05.2025_20:58:40 at: http://localhost:5000/#/experiments/3/runs/87c710ab940b4f3e9080c8d7c170e61e
🧪 View experiment at: http://localhost:5000/#/experiments/3


[I 2025-05-13 20:58:55,582] Trial 2 finished with value: 0.26 and parameters: {'objective': 'CrossEntropy', 'colsample_bylevel': 0.08264827862361696, 'depth': 6, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bernoulli'}. Best is trial 2 with value: 0.26.


🏃 View run run_13.05.2025_20:58:48 at: http://localhost:5000/#/experiments/3/runs/b46cce8d2bb14534b40961190bb82df3
🧪 View experiment at: http://localhost:5000/#/experiments/3


[I 2025-05-13 20:59:03,312] Trial 3 finished with value: 0.18 and parameters: {'objective': 'Logloss', 'colsample_bylevel': 0.06405782107884113, 'depth': 11, 'boosting_type': 'Plain', 'bootstrap_type': 'Bernoulli'}. Best is trial 3 with value: 0.18.


🏃 View run run_13.05.2025_20:58:55 at: http://localhost:5000/#/experiments/3/runs/da15f01e058e432f994c6bc7d3771055
🧪 View experiment at: http://localhost:5000/#/experiments/3


2025/05/13 20:59:07 WARNING mlflow.utils.requirements_utils: Found pandas version (2.1.4+dfsg) contains a local version label (+dfsg). MLflow logged a pip requirement for this package as 'pandas==2.1.4' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/05/13 20:59:07 WARNING mlflow.utils.requirements_utils: Found pandas version (2.1.4+dfsg) contains a local version label (+dfsg). MLflow logged a pip requirement for this package as 'pandas==2.1.4' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/05/13 20:59:07 WARNING mlflow.utils.requirements_utils: Found lz4 version (4.0.2+dfsg) contains a local version label (+dfsg). MLflow logged a pip requirement for this package as 'lz4==4.0.2' without the local version label to make it installa

🏃 View run params_opt_13.05.2025_20:58 at: http://localhost:5000/#/experiments/3/runs/ea02069264f8475a94a00efec43305f5
🧪 View experiment at: http://localhost:5000/#/experiments/3


## Wrap dataset with Giskard

In [ ]:
raw_data = pd.concat([X_test, y_test], axis = 1)
giskard_dataset = Dataset(
    df = raw_data,
    target=TARGET_COLUMN_NAME,
    name = "German credit scoring dataset",
    cat_columns=COLUMNS_TO_ENCODE
)

## Wrap model with Giskard

In [ ]:
giskard_model = Model(
    model=catboostclassifier,
    model_type="classification",     # Either regression, classification or text_generation.
    name="Chunk classification",
    classification_labels=catboostclassifier.classes_,  # Their order MUST be identical to the prediction_function's output order
    feature_names=FEATURE_COLUMNS     # Default: all columns of your dataset
)

## Scan model with Giskard

In [ ]:
results = scan(giskard_model, giskard_dataset, verbose=False)
results.to_html("giskard_scan_result.html")

# Save model

In [20]:
with open('model.pkl', 'wb') as f:
    pickle.dump(catboostclassifier, f)

# Minio

In [21]:
def s3_upload_model():
    # Create a client with the MinIO server playground, its access key
    # and secret key.
    client = Minio("localhost:9099",
        access_key=ACCESS_KEY,
        secret_key=SECRET_KEY,
        secure=False
    )

    # The file to upload, change this path if needed
    source_file = "model.pkl"

    # The destination bucket and filename on the MinIO server
    bucket_name = "credit-model"
    destination_file = "model.pkl"

    # Make the bucket if it doesn't exist.
    found = client.bucket_exists(bucket_name)
    if not found:
        client.make_bucket(bucket_name)
        print("Created bucket", bucket_name)
    else:
        print("Bucket", bucket_name, "already exists")

    # Upload the file, renaming it in the process
    client.fput_object(
        bucket_name, destination_file, source_file,
    )
    print(
        source_file, "successfully uploaded as object",
        destination_file, "to bucket", bucket_name,
    )

try:
    s3_upload_model()
except S3Error as exc:
    print("error occurred.", exc)

Bucket credit-model already exists
model.pkl successfully uploaded as object model.pkl to bucket credit-model
